Dataset with target value in json



## 1. Custom Data Loader


In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

In [ ]:
class BrainTumorDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        """
        Args:
            img_dir (str): Directory with all the images.
            transform (callable, optional): Optional transformations to apply to the images.
        """
        self.img_dir = img_dir
        self.transform = transform
        self.image_files = os.listdir(img_dir)  # List all image files in the directory

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Get image path
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        
        # Load image
        image = Image.open(img_path).convert("RGB")
        
        # Extract label from filename
        if "tumor" in img_name.lower():
            label = 1
        elif "no_tumor" in img_name.lower():
            label = 0
        else:
            raise ValueError(f"Cannot infer label from filename: {img_name}")
        
        # Apply transformations
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [ ]:
# Path to images
img_dir = 'brain_tumor_dataset/images'

# Data transformations for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the custom dataset
full_dataset = BrainTumorDataset(img_dir=img_dir, transform=data_transforms['train'])

# Split into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Set the correct transform for each dataset
train_dataset.dataset.transform = data_transforms['train']  # Apply train transform to train split
val_dataset.dataset.transform = data_transforms['val']      # Apply val transform to validation split

# Create DataLoaders for training and validation
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Check the number of batches
print(f"Train loader batches: {len(train_loader)}")
print(f"Validation loader batches: {len(val_loader)}")

## 1. ImageWithTargetDataset In json een tagert 


In [1]:
import json
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim

In [2]:
class ImageWithTargetDataset(Dataset):
    def __init__(self, img_dir, annotations_file, transform=None):
        """
        Args:
            img_dir (str): Path to the directory with the images.
            annotations_file (str): Path to the JSON file with annotations.
            transform (callable, optional): Optional transformations to apply to the images.
        """
        self.img_dir = img_dir
        self.transform = transform
        # Load annotations from the JSON file
        with open(annotations_file, 'r') as f:
            self.annotations = json.load(f)
        # Create a list of image filenames
        self.image_files = list(self.annotations.keys())

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Get the filename
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        # Load the image
        image = Image.open(img_path).convert("RGB")
        # Get the target value (people count)
        target = self.annotations[img_name]["people_count"]
        # Apply transformations
        if self.transform:
            image = self.transform(image)
        return image, target

In [3]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [4]:
img_dir = 'path_to_images'  # Replace this with the actual path to your images
annotations_file = 'annotations.json'  # Path to your JSON file with annotations

# Load the full dataset
full_dataset = ImageWithTargetDataset(img_dir=img_dir, annotations_file=annotations_file, transform=data_transforms)

# Get dataset size and print
dataset_size = len(full_dataset)
print(f"Total number of samples in the dataset: {dataset_size}")

# Split the dataset into 80% train and 20% test
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create DataLoaders for train and test
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check batch sizes
print(f"Train loader batches: {len(train_loader)}")
print(f"Test loader batches: {len(test_loader)}")

In [5]:
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(224 * 224 * 3, 128)  # 224x224x3 = 150528 input features
        self.fc2 = nn.Linear(128, 1)  # 1 output: predicted people count

    def forward(self, x):
        x = x.view(-1, 224 * 224 * 3)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
model = RegressionModel()
criterion = nn.MSELoss()  # Mean Squared Error (MSE) for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))  # Loss expects a vector
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

## 1. Custom Data Generator with Keras `Sequence` Class
Using a custom generator class is helpful when handling large datasets that don't fit in memory. This approach loads data in batches.

In [ ]:
# Import required libraries
from tensorflow.keras.utils import Sequence
import numpy as np
import cv2

In [ ]:
# Define custom data generator class
class CustomDataGenerator(Sequence):
    def __init__(self, image_filenames, labels, batch_size, image_size):
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        # Load and normalize images
        images = [cv2.resize(cv2.imread(filename), self.image_size) for filename in batch_x]
        images = np.array(images) / 255.0
        labels = np.array(batch_y)

        return images, labels

In [ ]:
# Example usage of the CustomDataGenerator
image_filenames = ['path/to/image1.jpg', 'path/to/image2.jpg']  # Replace with actual paths
labels = [0, 1]  # Replace with actual labels

data_generator = CustomDataGenerator(image_filenames, labels, batch_size=32, image_size=(128, 128))

# Check the output shape
sample_images, sample_labels = data_generator[0]
print(sample_images.shape, sample_labels.shape)

## 2. Using `image_dataset_from_directory`
This function is suitable for structured datasets where images are organized in subdirectories for each class.

In [ ]:
# Import TensorFlow
import tensorflow as tf

In [ ]:
# Load dataset from directory
dataset_directory = "path/to/dataset"  # Replace with actual path
batch_size = 32
image_size = (128, 128)

train_dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_directory,
    labels="inferred",
    label_mode="int",
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True
)

In [ ]:
# Check the dataset by printing a batch of images and labels
for images, labels in train_dataset.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

## 3. Using `list_files` and `.map()`
For datasets with more complex structures or mixed data, you can use `list_files` to collect file paths and `.map()` to preprocess.

In [ ]:
# Define file pattern and parameters
file_pattern = "path/to/dataset/*/*.jpg"  # Replace with actual file pattern
batch_size = 32
image_size = (128, 128)

# Get file paths
file_paths = tf.data.Dataset.list_files(file_pattern, shuffle=True)

In [ ]:
# Define a function to load and preprocess each image
def load_and_preprocess_image(file_path):
    label = tf.strings.split(file_path, '/')[-2]  # Use folder name as label
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = image / 255.0  # Normalize
    return image, label

In [ ]:
# Apply the function using `.map()`
dataset = file_paths.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# Batch the dataset
dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)